In [15]:
from requests import get
from json import loads
from time import sleep
from typing import List
from borsdata_sdk.models.Instrument import Instrument
from borsdata_sdk.models.Market import Market
import pandas as pd
from http import HTTPStatus
from borsdata_sdk import BorsdataAPI
import borsdata_sdk
import requests
from pandas.io.json import json_normalize
import numpy as np

In [16]:
auth_key = 'f4e8e3c7c46c4c34ae238a1822863410'

RATE_LIMIT = 429
borsdata = BorsdataAPI(auth_key)
instruments = borsdata.get_instruments()

In [17]:
my_list = []
for i in range(len(instruments)):
    tmp_list = [instruments[i].insId,instruments[i].name,instruments[i].countryId,instruments[i].sectorId,instruments[i].marketId,instruments[i].instrument]
    my_list.append(tmp_list)

instruments_df=pd.DataFrame(my_list,columns=['insID','Name','countryId','sectorId', 'marketId', 'instrument']) 

# Rename the lists to the real names
# MarketId: 1 = Large Cap, 2 = Mid Cap, 3=Small Cap, 4=First North

swe_stock_main = instruments_df[(instruments_df['countryId'] == 1) & (instruments_df['instrument'] == 0) & (instruments_df['marketId'].isin([1,2,3, 4]))]

### TEST WITH JSON

- stack reports

In [18]:
URL = "https://apiservice.borsdata.se/v1/instruments/2/reports?authKey=f4e8e3c7c46c4c34ae238a1822863410&maxQuarterCount=40"
response = requests.get(URL)
#tmp_df = json_normalize(response.json(), record_path ='reportsR12')

In [25]:
#insID = 2
#swe_stock_main['insID'].values:

#REPORTS_df = pd.DataFrame(columns=['d', 'insID','h','l','c','o','v'])
#REPORTS_df = REPORTS_df.set_index(['d', 'insID'])
REPORTS_df = pd.DataFrame()

for ins in swe_stock_main['insID'].values:
    sleep(1) 
    
    q_URL = "https://apiservice.borsdata.se/v1/instruments/"+str(ins)+"/reports?authKey=f4e8e3c7c46c4c34ae238a1822863410&maxQuarterCount=40"
    y_URL = "https://apiservice.borsdata.se/v1/instruments/"+str(ins)+"/reports?authKey=f4e8e3c7c46c4c34ae238a1822863410&maxYearCount=40"
    r12_URL = "https://apiservice.borsdata.se/v1/instruments/"+str(ins)+"/reports?authKey=f4e8e3c7c46c4c34ae238a1822863410&maxR12Count=40"
    q_response = requests.get(q_URL)
    y_response = requests.get(y_URL)
    r12_response = requests.get(r12_URL)

    R12_tmp = json_normalize(r12_response.json(), record_path ='reportsR12')
    #if report date is missing, set first day of next year
    R12_tmp.loc[R12_tmp['report_Date'].isna(),'report_Date'] = pd.to_datetime(R12_tmp[R12_tmp['report_Date'].isna()]['year']+1,format="%Y")
    
    R12_tmp['report_Date'] = pd.to_datetime(R12_tmp['report_Date'])
    R12_tmp['d'] = pd.to_datetime(R12_tmp['report_Date'])
    R12_tmp['insID'] = ins
    R12_tmp = R12_tmp.set_index(['d', 'insID'])
    
    yearly_tmp = json_normalize(y_response.json(), record_path ='reportsYear')
    #if report date is missing, set first day of next year
    yearly_tmp.loc[yearly_tmp['report_Date'].isna(),'report_Date'] = pd.to_datetime(yearly_tmp[yearly_tmp['report_Date'].isna()]['year']+1,format="%Y")
    
    yearly_tmp['report_Date'] = pd.to_datetime(yearly_tmp['report_Date'])
    yearly_tmp['d'] = pd.to_datetime(yearly_tmp['report_Date'])
    yearly_tmp['insID'] = ins
    yearly_tmp = yearly_tmp.set_index(['d', 'insID'])
    
    quarter_tmp = json_normalize(q_response.json(), record_path ='reportsYear')
    #if report date is missing, set first day of next year
    quarter_tmp.loc[quarter_tmp['report_Date'].isna(),'report_Date'] = pd.to_datetime(quarter_tmp[quarter_tmp['report_Date'].isna()]['year']+1,format="%Y")
    
    quarter_tmp['report_Date'] = pd.to_datetime(quarter_tmp['report_Date'])
    quarter_tmp['d'] = pd.to_datetime(quarter_tmp['report_Date'])
    quarter_tmp['insID'] = ins
    quarter_tmp = quarter_tmp.set_index(['d', 'insID'])
    quarter_tmp.columns = quarter_tmp.columns + "_quarter"
    
    full_report_tmp = pd.concat([R12_tmp,yearly_tmp, quarter_tmp], sort=True)
    REPORTS_df = pd.concat([REPORTS_df,full_report_tmp], sort=True) 
    
REPORTS_df = REPORTS_df.sort_index() 

In [26]:
SAVE = 'YES'

if SAVE == 'YES':
    REPORTS_df.to_csv("SWE_REPORTS.csv")

In [21]:
URL = "https://apiservice.borsdata.se/v1/instruments/"+str(2336)+"/reports?authKey=f4e8e3c7c46c4c34ae238a1822863410&maxQuarterCount=40"
response = requests.get(URL)

R12_tmp = json_normalize(response.json(), record_path ='reportsR12')

In [123]:
R12_tmp

,year,period,revenues,gross_Income,operating_Income,profit_Before_Tax,profit_To_Equity_Holders,earnings_Per_Share,number_Of_Shares,dividend,...,stock_Price_Average,stock_Price_High,stock_Price_Low,report_Start_Date,report_End_Date,broken_Fiscal_Year,currency,currency_Ratio,net_Sales,report_Date
0,2020,4,0.0,0.0,0.0,0.0,0.0,0.0,53.702,0.0,...,7.05,7.6,6.5,2020-01-01T00:00:00,2020-12-31T00:00:00,False,SEK,1.0,0.0,None
1,2019,4,0.0,0.0,0.0,0.0,0.0,0.0,53.702,0.0,...,0.00,0.0,0.0,2019-01-01T00:00:00,2019-12-31T00:00:00,False,SEK,1.0,0.0,None


In [121]:
R12_tmp[R12_tmp['report_Date'].isna()]['report_Date'] = pd.to_datetime(R12_tmp[R12_tmp['report_Date'].isna()]['year']+1,format="%Y")

In [107]:
pd.to_datetime(R12_tmp[R12_tmp['report_Date'].isna()]['year']+1,format="%Y")

0   2021-01-01
1   2020-01-01
Name: year, dtype: datetime64[ns]

In [114]:
R12_tmp[R12_tmp['report_Date'].isna()]

year  period  revenues  gross_Income  operating_Income  \
d   insID                                                           
NaT 2286   2021       2   457.837       413.574           -95.618   
    2286   2020       4   438.103       392.962           -73.933   

           profit_Before_Tax  profit_To_Equity_Holders  earnings_Per_Share  \
d   insID                                                                    
NaT 2286            -109.711                  -107.360             -0.6961   
    2286             -86.250                   -84.831             -0.5500   

           number_Of_Shares  dividend  ...  stock_Price_Average  \
d   insID                              ...                        
NaT 2286            154.222       0.0  ...                  0.0   
    2286            154.222       0.0  ...                  0.0   

           stock_Price_High  stock_Price_Low    report_Start_Date  \
d   insID                                                           
NaT 2286                0.0              0.0  2020-07-01T00:00:00   
    2286                0.0              0.0  2020-01-01T00:00:00   

               report_End_Date  broken_Fiscal_Year  currency  currency_Ratio  \
d   insID                                                                      
NaT 2286   2021-06-30T00:00:00               False       SEK             1.0   
    2286   2020-12-31T00:00:00               False       SEK             1.0   

           net_Sales  report_Date  
d   insID                          
NaT 2286     364.529          NaT  
    2286     348.207          NaT  

[2 rows x 37 columns]

In [126]:
R12_tmp.loc[R12_tmp['report_Date'].isna(),'report_Date'] = pd.to_datetime(R12_tmp[R12_tmp['report_Date'].isna()]['year']+1,format="%Y")

In [124]:
R12_tmp.loc[R12_tmp['report_Date'].isna(),'report_Date']

0    None
1    None
Name: report_Date, dtype: object

In [125]:
pd.to_datetime(R12_tmp[R12_tmp['report_Date'].isna()]['year']+1,format="%Y")

0   2021-01-01
1   2020-01-01
Name: year, dtype: datetime64[ns]